# Suspicious Process Creation
Detects suspicious process starts on Windows systems based on keywords

## Rule Content
```
- title: Suspicious Process Creation
  id: 5f0f47a5-cb16-4dbe-9e31-e8d976d73de3
  description: Detects suspicious process starts on Windows systems based on keywords
  status: experimental
  references:
  - https://www.swordshield.com/2015/07/getting-hashes-from-ntds-dit-file/
  - https://www.youtube.com/watch?v=H3t_kHQG1Js&feature=youtu.be&t=15m35s
  - https://winscripting.blog/2017/05/12/first-entry-welcome-and-uac-bypass/
  - https://twitter.com/subTee/status/872244674609676288
  - https://docs.microsoft.com/en-us/windows-hardware/drivers/debugger/remote-tool-examples
  - https://tyranidslair.blogspot.ca/2017/07/dg-on-windows-10-s-executing-arbitrary.html
  - https://www.trustedsec.com/2017/07/new-tool-release-nps_payload/
  - https://subt0x10.blogspot.ca/2017/04/bypassing-application-whitelisting.html
  - https://gist.github.com/subTee/7937a8ef07409715f15b84781e180c46#file-rat-bat
  - https://twitter.com/vector_sec/status/896049052642533376
  - http://security-research.dyndns.org/pub/slides/FIRST-TC-2018/FIRST-TC-2018_Tom-Ueltschi_Sysmon_PUBLIC.pdf
  author: Florian Roth
  modified: 2018/12/11
  tags:
  - car.2013-07-001
  logsource:
    category: process_creation
    product: windows
    service: null
  detection:
    selection:
      CommandLine:
      - vssadmin.exe delete shadows*
      - vssadmin delete shadows*
      - vssadmin create shadow /for=C:*
      - copy \\?\GLOBALROOT\Device\\*\windows\ntds\ntds.dit*
      - copy \\?\GLOBALROOT\Device\\*\config\SAM*
      - reg SAVE HKLM\SYSTEM *
      - reg SAVE HKLM\SAM *
      - '* sekurlsa:*'
      - net localgroup administrators * /add
      - net group "Domain Admins" * /ADD /DOMAIN
      - certutil.exe *-urlcache* http*
      - certutil.exe *-urlcache* ftp*
      - netsh advfirewall firewall *\AppData\\*
      - attrib +S +H +R *\AppData\\*
      - schtasks* /create *\AppData\\*
      - schtasks* /sc minute*
      - '*\Regasm.exe *\AppData\\*'
      - '*\Regasm *\AppData\\*'
      - '*\bitsadmin* /transfer*'
      - '*\certutil.exe * -decode *'
      - '*\certutil.exe * -decodehex *'
      - '*\certutil.exe -ping *'
      - icacls * /grant Everyone:F /T /C /Q
      - '* wmic shadowcopy delete *'
      - '* wbadmin.exe delete catalog -quiet*'
      - '*\wscript.exe *.jse'
      - '*\wscript.exe *.js'
      - '*\wscript.exe *.vba'
      - '*\wscript.exe *.vbe'
      - '*\cscript.exe *.jse'
      - '*\cscript.exe *.js'
      - '*\cscript.exe *.vba'
      - '*\cscript.exe *.vbe'
      - '*\fodhelper.exe'
      - '*waitfor*/s*'
      - '*waitfor*/si persist*'
      - '*remote*/s*'
      - '*remote*/c*'
      - '*remote*/q*'
      - '*AddInProcess*'
      - '* /stext *'
      - '* /scomma *'
      - '* /stab *'
      - '* /stabular *'
      - '* /shtml *'
      - '* /sverhtml *'
      - '* /sxml *'
    condition: selection
  falsepositives:
  - False positives depend on scripts and administrative tools used in the monitored
    environment
  level: medium

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='process_command_line.keyword:(vssadmin.exe\ delete\ shadows* OR vssadmin\ delete\ shadows* OR vssadmin\ create\ shadow\ \/for\=C\:* OR copy\ \\?\\GLOBALROOT\\Device\\*\\windows\\ntds\\ntds.dit* OR copy\ \\?\\GLOBALROOT\\Device\\*\\config\\SAM* OR reg\ SAVE\ HKLM\\SYSTEM\ * OR reg\ SAVE\ HKLM\\SAM\ * OR *\ sekurlsa\:* OR net\ localgroup\ administrators\ *\ \/add OR net\ group\ \"Domain\ Admins\"\ *\ \/ADD\ \/DOMAIN OR certutil.exe\ *\-urlcache*\ http* OR certutil.exe\ *\-urlcache*\ ftp* OR netsh\ advfirewall\ firewall\ *\\AppData\\* OR attrib\ \+S\ \+H\ \+R\ *\\AppData\\* OR schtasks*\ \/create\ *\\AppData\\* OR schtasks*\ \/sc\ minute* OR *\\Regasm.exe\ *\\AppData\\* OR *\\Regasm\ *\\AppData\\* OR *\\bitsadmin*\ \/transfer* OR *\\certutil.exe\ *\ \-decode\ * OR *\\certutil.exe\ *\ \-decodehex\ * OR *\\certutil.exe\ \-ping\ * OR icacls\ *\ \/grant\ Everyone\:F\ \/T\ \/C\ \/Q OR *\ wmic\ shadowcopy\ delete\ * OR *\ wbadmin.exe\ delete\ catalog\ \-quiet* OR *\\wscript.exe\ *.jse OR *\\wscript.exe\ *.js OR *\\wscript.exe\ *.vba OR *\\wscript.exe\ *.vbe OR *\\cscript.exe\ *.jse OR *\\cscript.exe\ *.js OR *\\cscript.exe\ *.vba OR *\\cscript.exe\ *.vbe OR *\\fodhelper.exe OR *waitfor*\/s* OR *waitfor*\/si\ persist* OR *remote*\/s* OR *remote*\/c* OR *remote*\/q* OR *AddInProcess* OR *\ \/stext\ * OR *\ \/scomma\ * OR *\ \/stab\ * OR *\ \/stabular\ * OR *\ \/shtml\ * OR *\ \/sverhtml\ * OR *\ \/sxml\ *)')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()